In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [8]:
dataset = 'emma' # onion or emma
results_dir = f'../results/' + dataset + '/performance/'
cutoffs = [1, 5, 10, 20]
metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR', 'EFD', 'EPC', 'ARP', 'PopREO', 'PopRSP', 'ItemCoverage', 'UserCoverage']
#metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR']

In [9]:
def find_modality(string):
    keyword = "modalites="
    start = string.find(keyword)
    if start == -1:
        return None

    start += len(keyword)
    end = string.find("_", start)
    if end == -1:
        end = len(string)

    modality = string[start:end].strip("'")
    return modality

In [10]:
# read all files in result folder
files = os.listdir(results_dir)
df = pd.DataFrame()

for cutoff in cutoffs:
    df_cut = pd.DataFrame()
    result_files_cutoff = [f for f in files if f.startswith(f'rec_cutoff_{cutoff}_')]
    for f in result_files_cutoff:
        df_tmp = pd.read_csv(results_dir + f, sep='\t')
        tmp_metrics = [m for m in metrics if m in df_tmp.columns]
        df_tmp.rename(columns={m: f'{m}@{cutoff}' for m in tmp_metrics}, inplace=True)
        df_tmp['modality'] = find_modality(df_tmp['model'].values[0])
        df_tmp['model'] = df_tmp['model'].str.split('_').str[0]
        df_tmp['date'] = f[-23:-4]
        df_cut = pd.concat([df_cut, df_tmp], axis=0)

    # keep only newest model results
    df_cut = df_cut.sort_values('date').drop_duplicates(subset=['model', 'modality'], keep='last')

    # merge with previous cutoffs
    if not df.empty:
        df = pd.merge(df, df_cut, on=['model', 'date', 'modality'], how='inner')
        print(df.shape)
    else:
        df = df_cut

df = df[['model', 'modality', 'date'] + [c for c in df.columns if '@' in c]]
df.sort_values('nDCG@5', inplace=True)
# display columns with @10 only
#df[['model', 'modality', 'date'] + [c for c in df.columns if '@1' in c]]
df[['model', 'modality', 'date'] + ['Recall@1', 'Recall@5', 'Recall@10', 'nDCG@1', 'nDCG@5', 'nDCG@10']]

(47, 27)
(47, 39)
(47, 51)


,model,modality,date,Recall@1,Recall@5,Recall@10,nDCG@1,nDCG@5,nDCG@10
31,Random,None,2025_01_16_10_22_24,0.004789,0.025369,0.049089,0.009534,0.017747,0.026635
1,NeuMF,None,2025_01_15_11_51_31,0.076523,0.177315,0.247935,0.143273,0.158442,0.184691
10,MultiVAE,None,2025_01_15_14_20_14,0.078494,0.270344,0.403570,0.150159,0.208182,0.259350
7,GRCN,emotion,2025_01_15_13_27_44,0.090612,0.281955,0.408933,0.163930,0.222449,0.271326
14,GRCN,audio,2025_01_15_15_06_11,0.092786,0.304825,0.444438,0.172669,0.237971,0.291832
26,GRCN,textual,2025_01_15_17_37_47,0.093562,0.305874,0.442527,0.175847,0.239282,0.291426
38,BiVAECFM,visual,2025_02_04_13_24_34,0.122661,0.291617,0.440381,0.222458,0.247947,0.304279
9,MMGCN,emotion,2025_01_15_14_05_53,0.126079,0.299171,0.429137,0.229873,0.252230,0.301410
36,BiVAECFM,emotion,2025_02_04_10_19_08,0.123916,0.298042,0.424120,0.226430,0.252261,0.299759
2,BiVAECF,None,2025_01_15_12_39_33,0.123488,0.300994,0.432495,0.224311,0.252474,0.302488


In [13]:
# export to comma seperated file
df.to_csv(f'../results/{dataset}_performance.csv', index=False)

In [14]:
for cutoff in cutoffs:
    df_cut = df[['model'] + [c for c in df.columns if f'@{cutoff}' in c]]
    df_cut.to_csv(f'../results/{dataset}_performance_{cutoff}.csv', index=False)